In [1]:
import torch

In [2]:
SEED = 1
torch.manual_seed(SEED)
import random
random.seed(SEED)

In [3]:
from igre_dqn_v3 import IgreDQNTrainer
from igre_v2 import Igre

/Users/mocuto/anaconda3/envs/m2/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/Users/mocuto/anaconda3/envs/m2/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/Users/mocuto/anaconda3/envs/m2/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import AutoTokenizer
from based.models.gpt import GPTLMHeadModel

tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = GPTLMHeadModel.from_pretrained_hf("hazyresearch/based-360m")

based_model = model.transformer
lm_head = model.lm_head

ModuleNotFoundError: No module named 'based'

# Load Sim4 data

In [ ]:
import json
import pickle

In [ ]:

sim_4_training_data_unpickled = pickle.load(open("sim4_data2.pkl", "rb"))
len(sim_4_training_data_unpickled)

In [ ]:
sim_4_training_data_unpickled[1].actor

# Training

In [ ]:
model = Igre(based_model, lm_head, tokenizer)
model.cuda()
model.train()

In [ ]:
DEBUG_NAN = True
def check_activation_for_nan(name):
    def hook(model, input, output):
        if type(output) is tuple:
            for i, out in enumerate(output):
                if type(out) is torch.Tensor and torch.isnan(out).any():
                    print(f"Found NaN in {name}[{i}]")
        elif type(output) is torch.Tensor:
            if torch.isnan(output).any():
                print(f"Found NaN in {name}")
    return hook
if DEBUG_NAN:
    for name, module in model.named_modules():
        module.register_forward_hook(check_activation_for_nan(name))

In [ ]:
dqn_trainer = IgreDQNTrainer(model)

In [ ]:
import random
import re

teacher_forcing_count = 0

def get_completion(prompt):
    global teacher_forcing_count
    prompt_ids = tokenizer.encode(prompt, return_tensors="pt").cuda()
    response_ids, z = dqn_trainer.get_response(prompt_ids)
    metadata = {
        "z": z,
        "response_ids": response_ids.detach().cpu()
    }
    # use_teacher_forcing = random.random() < 0.5
    # if use_teacher_forcing and teacher_forcing_count < 100:
    #     teacher_forcing_count += 1
    #     print("use teacher forcing")
    #     finish_X_re = re.compile(" finish ([ABCD])")
    #     match = finish_X_re.search(prompt)
    #     if match:
    #         finish_X = match.group(0).strip()
    #         return finish_X, metadata
    print("completion response ids")
    print(response_ids)
    response = tokenizer.decode(response_ids.squeeze(0), skip_special_tokens=True)
    # if '"' in response or "'" in response:
    #     # Remove quotes
    #     response = response.replace('"', "")
    #     response = response.replace("'", "")
    #     response_ids = tokenizer.encode(response, return_tensors="pt").cuda()
    return tokenizer.decode(response_ids.squeeze(0), skip_special_tokens=True), metadata

In [ ]:
get_completion("test")

In [ ]:
from simulation.sim_runner import SimRunner
from simulation.common.base_actor import AIActor

In [ ]:
def process_transition(transition, finetune=False, finetune_state_to_prompt_fn=None):
    assert finetune == False or (finetune == True and finetune_state_to_prompt_fn is not None)
    def inner(prompt, response, next_state_prompt):
        prompt_ids = tokenizer.encode(prompt, return_tensors="pt")
        response_ids = tokenizer.encode(
            response, return_tensors="pt", add_special_tokens=False
        ).long()
        # Add eos token to response_ids
        response_ids = torch.cat([response_ids, torch.tensor([[tokenizer.eos_token_id]], dtype=torch.long)], dim=1).long()  # type: ignore

        next_state_prompt_ids = tokenizer.encode(
            next_state_prompt, return_tensors="pt", add_special_tokens=True
        ).long()

        act_metadata = transition.action.metadata
        z = torch.zeros(1)
        if act_metadata is not None and "z" in act_metadata:
            z = act_metadata["z"]
        if act_metadata is not None and "response_ids" in act_metadata:
            response_ids = act_metadata["response_ids"]
            response_ids = torch.cat([response_ids, torch.tensor([[tokenizer.eos_token_id]], dtype=torch.long)], dim=1).long()  # type: ignore

        reward = (transition.next_state.score - transition.state.score) / 100
        reward = torch.tensor([reward]).float().detach()

        next_state_is_terminal = torch.tensor([1. if transition.next_state.is_terminal else 0.])

        print(f"prompt: {prompt}")
        print(f"response: {response}")
        dqn_trainer.add_transition(
            prompt_ids.squeeze(0),
            response_ids.squeeze(0),
            z,
            reward,
            next_state_prompt_ids.squeeze(0),
            next_state_is_terminal,
            add_to_finetune_memory=finetune,
        )
    if finetune:
        # Check the classname for "player"
        if "player" in transition.actor.__class__.__name__.lower():
            assert finetune_state_to_prompt_fn is not None
            prompt = finetune_state_to_prompt_fn(transition.state)
            response = transition.action.content
            next_state_prompt = finetune_state_to_prompt_fn(transition.next_state)
            inner(prompt, response, next_state_prompt)
    else:
        if isinstance(transition.actor, AIActor) and transition.actor.name == "player":
            prompt = transition.action.completion_prompt
            response = transition.action.completion_response

            print("## DEBUG prompt")
            print(prompt)
            prompt_ids = tokenizer.encode(prompt, return_tensors="pt")
            print(prompt_ids)

            next_state = transition.next_state
            next_state_prompt = transition.actor.state_to_prompt(next_state)
            inner(prompt, response, next_state_prompt)

        # prompt_ids = tokenizer.encode(prompt, return_tensors="pt")
        # response_ids = tokenizer.encode(
        #     response, return_tensors="pt", add_special_tokens=False
        # ).long()
        # # Add eos token to response_ids
        # response_ids = torch.cat([response_ids, torch.tensor([[tokenizer.eos_token_id]], dtype=torch.long)], dim=1).long()  # type: ignore

        # next_state_prompt_ids = tokenizer.encode(
        #     next_state_prompt, return_tensors="pt", add_special_tokens=False
        # ).long()

        # z = transition.action.metadata["z"]
        # reward = transition.next_state.score - transition.state.score
        # reward = torch.tensor([reward])
        # print(f"prompt: {prompt}")
        # print(f"response: {response}")
        # dqn_trainer.add_transition(
        #     prompt_ids.squeeze(0),
        #     response_ids.squeeze(0),
        #     z,
        #     reward,
        #     next_state_prompt_ids.squeeze(0),
        #     transition.next_state.is_terminal,
        # )

In [ ]:
sim_name = "sim4"
sim_runner = SimRunner(
    sim_name=sim_name,
    seed=1,
    model_forward_func_for_ai={
        "player": get_completion,
    },
)

## Process transitions using the player actor in the sim

In [ ]:
ai_actor = None
for actor in sim_runner.sim.actors:
    if isinstance(actor, AIActor):
        ai_actor = actor
        break
ai_actor

In [ ]:
dqn_trainer.reset_memory(finetune=True)
for x in sim_4_training_data_unpickled:
    process_transition(x, finetune=True, finetune_state_to_prompt_fn=ai_actor.state_to_prompt)
    process_transition(x, finetune=True, finetune_state_to_prompt_fn=ai_actor.state_to_prompt)
    process_transition(x, finetune=True, finetune_state_to_prompt_fn=ai_actor.state_to_prompt)
len(dqn_trainer.finetune_memory)

In [ ]:


def run():
    sim_runner.reset(seed=1)
    print("## DEBUG sim run len")
    print(len(sim_runner.history))
    while not sim_runner.get_state().is_terminal:
        transitions = sim_runner.step()
        for x in transitions:
            if isinstance(x.actor, AIActor) and x.actor.name == "player":
                process_transition(x)
    print("done")
    final_transitions = sim_runner.final()
    did_win = False
    for x in final_transitions:
        process_transition(x)
        if x.next_state.score > 0:
            did_win = True
    return did_win

run()

In [ ]:
tokenizer.eos_token_id

In [ ]:
import matplotlib.pyplot as plt
from IPython import display
import time

In [ ]:
%matplotlib inline

def pltsin(ax, fig, x, y, hdisplay):
    if ax.lines:
        for line in ax.lines:
            line.set_xdata(x)
            line.set_ydata(y)
            # update scale
            ax.relim()
            ax.autoscale_view()

    else:
        ax.plot(x, y)
    hdisplay.update(fig)
    time.sleep(1)
fig, ax = plt.subplots(1, 1)
hdisplay = display.display("", display_id=True)
ax.set_label("game_count")
ax.set_label("win_rate")

In [ ]:
tokenizer.encode("finish x = 1")

In [ ]:
tokenizer.encode("finish x = 1", add_special_tokens=True)

In [ ]:
tokenizer.eos_token_id

In [ ]:
for i in range(7):
  #  remov
  if i < 3:
    continue
  # Train on the existing teacher data
  dqn_trainer.update(1.0, i == 0)

In [ ]:
i

In [ ]:
prompt = f"""
You are to solve the an algebraic expression.
You have access to two commands:
- 'calc <expression>' to calculate the value of an expression. <expression> must be in the form 'a + b' where a and b are integers.
- 'finish <solution>' to submit your answer. <solution> must be in the form '<letter> = <number>' where <letter> is the letter used in the problem and <number> is the solution.
The following is the algebraic expression:
x = 10 + 10
What is the value of x?
When you have an answer, use the 'finish' command to submit your answer.




player:"""

get_completion(prompt)

In [ ]:

game_count = 1200

results = []
xs = []
ys = []

for i in range(game_count):
    did_win = run()
    results.append(int(did_win))
    if i > 5:
        win_rate = sum(results[-5:]) / 5
        xs.append(i)
        ys.append(win_rate)
        pltsin(ax, fig, xs, ys, hdisplay)
    if i % 16 == 0:
        # lr_mult = 1.0 if i < 200 else 0.1
        lr_mult = 0.1 if i < 500 else 0.01 if i < 1000 else 0.001
        dqn_trainer.update(lr_mult, early_games = False)
    if i % 300 == 0:
        dqn_trainer.reset_memory(finetune=False)
    if i % 25 == 0:
        display.clear_output(wait=True)
    if i % 100 == 0:
        torch.save(model.state_dict(), f"model_dqn_{i}.pt")
        print(f"model saved at {i}")
        dqn_trainer.reset_memory()

In [ ]:
eval_game_count = 100
eval_results = []

teacher_forcing_count = 9999999999999999999

for i in range(eval_game_count):
    did_win = run()
    eval_results.append(int(did_win))
    if i % 25 == 0:
        display.clear_output(wait=True)

sum(eval_results) / eval_game_count

In [ ]:
prompt = f"""
You are to solve the an algebraic expression.
You have access to two commands:
- 'calc <expression>' to calculate the value of an expression. <expression> must be in the form 'a + b' where a and b are integers.
- 'finish <solution>' to submit your answer. <solution> must be in the form '<letter> = <number>' where <letter> is the letter used in the problem and <number> is the solution.
The following is the algebraic expression:
x = 23 + 93
What is the value of x?
When you have an answer, use the 'finish' command to submit your answer.




player:"""

get_completion(prompt)
# prompt_ids = tokenizer.encode(prompt, return_tensors="pt")
# prompt_ids